In [11]:
import random
import numpy as np
import os
import matplotlib.pyplot as plt
from dataset import dataset

print("testing...")

def make_plots(zero_one_list, f1_score_list, names_list):
    f1_data = [f1_scores[i] for i in range(len(f1_scores))]
    loss_data = [loss_scores[i] for i in range(len(loss_scores))]

    positions = np.arange(len(f1_scores))
    width = 0.4

    plt.figure(figsize=(8, 6))
    plt.boxplot(f1_data, positions=positions, widths=width, patch_artist=True,
                boxprops=dict(facecolor='lightblue'), medianprops=dict(color='blue'),
                whiskerprops=dict(color='blue'), capprops=dict(color='blue'),
                flierprops=dict(markerfacecolor='blue', marker='o'))
    plt.xticks(positions, names_list)
    plt.xlabel('Datasets')
    plt.ylabel('F1 Score')
    plt.title('F1 Scores Across Datasets')
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(8, 6))
    plt.boxplot(loss_data, positions=positions, widths=width, patch_artist=True,
                boxprops=dict(facecolor='lightcoral'), medianprops=dict(color='red'),
                whiskerprops=dict(color='red'), capprops=dict(color='red'),
                flierprops=dict(markerfacecolor='red', marker='o'))
    plt.xticks(positions, names_list)
    plt.xlabel('Datasets')
    plt.ylabel('0/1 Loss Score')
    plt.title('0/1 Loss Scores Across Datasets')
    plt.tight_layout()
    plt.show()
    

testing...


In [43]:
class algorithm:
    def __init__(self, data_array:dataset, which_data:str):
        # The algorithm takes in a pre-processed dataset object, there shouldn't be much to do with the constructor except maybe a setter for the dataset
        self.data_array = data_array

        # Determines label type
        cancer_labels = ['2','4']
        glass_labels = ['1','2','3','4','5','6','7']
        iris_labels = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
        soybean_labels = ['D1','D2','D3','D4']
        votes_labels = ['republican','democrat']
        if (which_data == 'cancer'):
            self.labels = cancer_labels
        elif (which_data == 'glass'):
            self.labels = glass_labels
        elif (which_data == 'iris'):
            self.labels = iris_labels
        elif (which_data == 'soybean'):
            self.labels = soybean_labels
        elif (which_data == 'votes'):
            self.labels = votes_labels

    def train(self):
        # Runs Q(), Runs F().

        # Q() calculates the class prior probabilities, this create be an array 10 * the # of classes in the datset (10 for each validation set).
        self.class_counts = [0] * len(self.labels)
        self.probs = [0] * len(self.labels)
        self.example_count = 0
        self.class_probs = []
        for i in range(10):
            self.rotated_data = self.data_array.partitions[i:] + self.data_array.partitions[:i]
            self.rotated_data = self.rotated_data[:-1]
            for partition in range(9):
                for example in range(len(self.rotated_data[partition])):
                    self.example_count += 1
                    for label in range(len(self.labels)):
                        if (self.rotated_data[partition][example][-1] == self.labels[label]):
                            self.class_counts[label] += 1
            for i, count in enumerate(self.class_counts):
                self.probs[i] = count/(self.example_count)
            self.class_probs.append(self.probs.copy())

        # F() calculates the individual attribute probabilities for each class. This creates an array of 10 * # of classes * # of attributes * # of attribute possibilites.
        

        # Once everything is run we should have trained 10 models for this method call. We will probably want to store each of these models in an array containing the other 2 arrays produced from Q and F, meaning there will be a ~7D array :/
        # We will also want to save each of the models for both validation purposes, and for the event processing fails downstream we won't have to re-train
        return
        
    def validate(self):

        fold_order = [9] + list(range(0, 9))  #sets the order for which partition is our validation set
        self.answers = []
        self.predictions = []
        for i in range(10):  #goes through each model
            test_fold_index = fold_order[i]
            class_prob_index = fold_order[(i + 1) % 10]

        
            test_fold = self.data_array.partitions[test_fold_index]
            class_probs = self.class_probs[class_prob_index]
            attribute_probs = self.attribute_probs[class_prob_index]
            
            self.correct_preditions = 0
            total_predictions = len(test_fold)

            for example in test_fold:
                features = example[:-1]
                true_label = example[-1]
                self.answers.append(true_label)
                max_class_prob = -float('inf')
                predicted_class = None

                for label in range(len(self.labels)):
                    class_prob = class_prob[label_index]

                    for feature_index, feature_value in enumerate(features):
                        feature_prob = attribute_probs[label_index][feature_index].get(feature_value, 1)
                        class_prob *= feature_prob


                    if class_prob > max_class_prob:
                        max_class_prob = class_prob
                        predicted_class = self.labels[label_index]

                predicted_label = predicted_class
                self.predictions.append(predicted_label)

              

                
            





        
        # This method runs the algorithm on the validation folds. We want to make sure we are running the proper model on the proper fold, so if you trained on folds 1-9, you would start the for loop at 10 and decrement (for example).
        # This will produce 10 classified folds, where we will want to save the results from the classification.
        return
    def calculate_loss(self):
        # This method will analyze the classification, and determine, TP, TN, FP, FN, F1 loss, 0/1 loss, etc.
        self.zero_one_losses = []
        self.f1_scores = []
        fold_order = [9] + list(range(0, 9))
        for fold_index in fold_order:
            test_fold = self.data_array.partitions[fold_index]

            true_positives = {label: 0 for label in self.labels}
            false_positives = {label: 0 for label in self.labels}
            false_negatives = {label: 0 for label in self.labels}
            correct_predictions = 0
            total_predictions = len(test_fold)


            for i in range(total_predictions):
                true_label = self.answers[i]
                predicted_label = self.predictions[i]


                if predicted_label == true_label:
                    correct_predictions += 1

                if predicted_label == true_label:
                    true_positives[true_label] += 1
                else:
                    false_positives[predicted_label] += 1
                    false_negatives[true_label] += 1
                

            loss = 1 - (correct_predictions / total_predictions)
            self.zero_one_loss.append(loss)



            f1_scores_per_class = []
            for label in self.labels:
                precision = true_positives[label] / (true_positives[label] + false_positives[label]) if (true_positives[label] + false_positives[label]) > 0 else 0
                recall = true_positives[label] / (true_positives[label] + false_negatives[label]) if (true_positives[label] + false_negatives[label]) > 0 else 0
                f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
                f1_scores_per_class.append(f1)
            
            average_f1 = sum(f1_scores_per_class) / len(self.labels)
            self.f1_scores.append(average_f1)












        
        # These results need to be saved
        # We will also want to create a box plot where the x-axis is the dataset we worked with (10 datasets), and y axis is a performance metric (whatever we want). For each dataset there will be 10 datapoints in the box, each point represents the performance of one of the folds.
        return


In [87]:
cancer_nopre = dataset('/home/carlos/Machine_Learning_Practice/processed_data/cancer_nopre.csv', 'last', True)
glass_nopre = dataset('/home/carlos/Machine_Learning_Practice/processed_data/glass_nopre.csv', 'last', True)
votes_nopre = dataset('/home/carlos/Machine_Learning_Practice/processed_data/votes_nopre.csv', 'first', True)
iris_nopre = dataset('/home/carlos/Machine_Learning_Practice/processed_data/iris_nopre.csv', 'last', True)
soybean_nopre = dataset('/home/carlos/Machine_Learning_Practice/processed_data/soybean_nopre.csv', 'last', True)

cancer_pre = dataset('/home/carlos/Machine_Learning_Practice/processed_data/cancer_pre.csv', 'last', True)
glass_pre = dataset('/home/carlos/Machine_Learning_Practice/processed_data/glass_pre.csv', 'last', True)
votes_pre = dataset('/home/carlos/Machine_Learning_Practice/processed_data/votes_pre.csv', 'first', True)
iris_pre = dataset('/home/carlos/Machine_Learning_Practice/processed_data/iris_pre.csv', 'last', True)
soybean_pre = dataset('/home/carlos/Machine_Learning_Practice/processed_data/soybean_pre.csv', 'last', True)

In [88]:
cancer_nopre_processing = algorithm(cancer_nopre, 'cancer')
cancer_nopre_processing.train()
print(f"Class Probabilities: {cancer_nopre_processing.class_probs}")
cancer_pre_processing = algorithm(cancer_pre, 'cancer')
cancer_pre_processing.train()
print(f"Class Probabilities: {cancer_pre_processing.class_probs}")

Class Probabilities: [[0.6507936507936508, 0.3492063492063492], [0.647339158061954, 0.3526608419380461], [0.6514830508474576, 0.3485169491525424], [0.653953118792213, 0.34604688120778704], [0.6525746980292435, 0.3474253019707565], [0.6535099337748345, 0.34649006622516554], [0.6537238873751136, 0.34627611262488645], [0.6554738724418836, 0.34452612755811646], [0.654892264217591, 0.34510773578240905], [0.6552217453505007, 0.3447782546494993]]
Class Probabilities: [[0.653968253968254, 0.346031746031746], [0.653693407466243, 0.34630659253375695], [0.6546610169491526, 0.3453389830508475], [0.656336909018673, 0.34366309098132697], [0.6560712015257469, 0.343928798474253], [0.6572185430463576, 0.34278145695364237], [0.6566757493188011, 0.34332425068119893], [0.655076495132128, 0.34492350486787204], [0.6555987283645355, 0.3444012716354645], [0.6552217453505007, 0.3447782546494993]]


In [ ]:
zero_one_list = []
